# OR 2-2, something with football i guess

In [43]:
import numpy as np
import pandas as pd

import os

In [82]:
data_files = os.listdir("data")
data = pd.read_csv(os.path.join("data", data_files[0]))

for data_file in data_files[1:]:
    data = pd.concat([data, pd.read_csv(os.path.join("data", data_file))])

data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,N1,29/08/08,Vitesse,Groningen,0.0,4.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N1,30/08/08,Nijmegen,Graafschap,2.0,0.0,H,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N1,30/08/08,Roda JC,Twente,1.0,1.0,D,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N1,30/08/08,Utrecht,PSV Eindhoven,1.0,5.0,A,0.0,3.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N1,30/08/08,Willem II,Ajax,2.0,1.0,H,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,N1,06/05/12,Heracles,Nijmegen,1.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,N1,06/05/12,NAC Breda,Waalwijk,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,N1,06/05/12,Roda,Utrecht,1.0,3.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,N1,06/05/12,Vitesse,Ajax,1.0,3.0,A,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
scores = data.loc[:, ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "Date"]].set_index(["HomeTeam", "AwayTeam"]).rename({"FTHG": "HomeGoals", "FTAG": "AwayGoals", "FTR": "HomeWin"}, axis=1)
scores["HomeWin"] = scores["HomeWin"] == "H"
scores = scores.dropna()
new_index = list(scores.index)
for i, v in enumerate(scores.iterrows()):
    clubs = v[0]
    res = v[1]
    if clubs[0] > clubs[1]:
        new_index[i] = (clubs[1], clubs[0])
        scores.iloc[i] = (res["AwayGoals"], res["HomeGoals"], not res["HomeWin"], res["Date"])

scores.index = pd.MultiIndex.from_tuples(new_index)
scores = scores.rename({"HomeGoals": "FirstGoals", "AwayGoals": "SecondGoals", "HomeWin": "FirstWin"}, axis=1)
scores

,,FirstGoals,SecondGoals,FirstWin,Date
Groningen,Vitesse,4.0,0.0,True,29/08/08
Graafschap,Nijmegen,0.0,2.0,False,30/08/08
Roda JC,Twente,1.0,1.0,False,30/08/08
PSV Eindhoven,Utrecht,5.0,1.0,True,30/08/08
Ajax,Willem II,1.0,2.0,False,30/08/08
...,...,...,...,...,...
Heracles,Nijmegen,1.0,2.0,False,06/05/12
NAC Breda,Waalwijk,3.0,2.0,True,06/05/12
Roda,Utrecht,1.0,3.0,False,06/05/12
Ajax,Vitesse,3.0,1.0,True,06/05/12


We can define the weights as follows:
$f_{i, j} = \frac{l_{i, j}}{2g_{i, j}} + \frac{c_{i, j}}{2c_{i, j} + 2s_{i, j}}$,
where $l_{i, j}$ is games lost, $g_{i, j}$  is total games. $c_{i, j}$ is the total goals conceded, $s_{i, j}$ is total goals scored. So first we need to get all those statistics. Note: $f_{i, j} + f_{j, i} = 1$.

In [84]:
stats = scores.groupby(scores.index).sum()
number_of_games = scores.groupby(scores.index).size().to_frame("GamesNumber")
stats = stats.merge(number_of_games, left_index=True, right_index=True)
stats.index = pd.MultiIndex.from_tuples(stats.index)
stats

FirstGoals  SecondGoals  FirstWin  GamesNumber
AZ Alkmaar Ajax             30.0         56.0        11           27
           Cambuur          14.0          5.0         7            8
           Den Haag         58.0         23.0        20           26
           Dordrecht         5.0          1.0         2            2
           Excelsior        25.0         21.0         8           14
...                          ...          ...       ...          ...
Vitesse    Zwolle           40.0         20.0        14           20
Volendam   Willem II         4.0          3.0         1            2
Waalwijk   Willem II         5.0         13.0         0            9
           Zwolle            8.0         13.0         4            9
Willem II  Zwolle           16.0         21.0         8           18

[369 rows x 4 columns]

In [86]:
weights = (stats["GamesNumber"] - stats["FirstWin"])/(2 * stats["GamesNumber"]) + (stats["SecondGoals"]/(2*stats["FirstGoals"] + 2*stats["SecondGoals"]))

for index, values in weights.copy():
    weights.append((index[1], index[0]), )

weights

AZ Alkmaar  Ajax         0.621878
            Cambuur      0.194079
            Den Haag     0.257360
            Dordrecht    0.083333
            Excelsior    0.442547
                           ...   
Vitesse     Zwolle       0.316667
Volendam    Willem II    0.464286
Waalwijk    Willem II    0.861111
            Zwolle       0.587302
Willem II   Zwolle       0.561562
Length: 369, dtype: float64